In [ ]:
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as sco
import requests
from sklearn.linear_model import LinearRegression as linreg
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ssga = "https://www.ssga.com/us/en/individual/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx"
spy_listings = list(pd.read_excel(ssga, header=4).Ticker.dropna())
data = yf.download(spy_listings, period="10y")
data = data.dropna(how="all", axis=1)
tickers = list(data['Close'])

In [ ]:
# prompt: count the number of nans in the dataframe data
data.isnull().sum().sum()

# prompt: code that removes all nans files across the axis
data = data.dropna(axis=0)

#check wuill confirm all nans are removed
data.isnull().sum().sum()

PRICE MOMENTUM FACTORS & IDENTIFYING LONG AND SHORT ASSETS

In [ ]:
class momentum_factors:
    def __init__(self, spy_listings, tickers):
        self.price, self.volume = spy_listings['Close'], spy_listings['Volume']
        self.returns = self.price.pct_change()
        self.tickers = tickers

        self.momentumdf = pd.DataFrame(index=self.tickers)

    def trend(self, input_df):
        self.input_df = input_df
        self.trend_df = pd.DataFrame(index=self.tickers)

        for i in range(1, 21):
            self.trend_df[i] = np.polyfit(
                range(252),
                self.input_df[-i-252: -i],
                1,
            )[0]

        return self.trend_df.mean(axis=1)

    def price_momentumfactors(self):

        # Run 52-Wk Trend Func.
        self.momentumdf["trend"] = self.trend(input_df=self.price)

        # Percent Above
        self.roll_min = self.price.rolling(252).min()
        self.momentumdf["pct_above"] = ((self.price - self.roll_min) / self.roll_min)[-20:].mean()

        # Price Osc.
        self.rolling_260 = self.price.rolling(260)
        self.momentumdf["p_osc"] = (
            (self.price.rolling(20).mean() - self.rolling_260.mean()) /
            self.rolling_260.std()
        )[-20:].mean()

        # 39-Wk Ret.
        self.momentumdf["39_wk_ret"] = self.price.pct_change(189)[-20:].mean()

        # Price Vol. Trend
        self.price_volumedf = self.price * self.volume
        self.momentumdf["price_volume"] = self.trend(input_df=self.price_volumedf)

        return ((self.momentumdf - self.momentumdf.mean()) / self.momentumdf.std()).sum(axis=1)

    def baskets(self):
          self.momentumscore = self.price_momentumfactors()
          self.z_scored = self.momentumscore

          return self.z_scored.nlargest(10).index, self.z_scored.nsmallest(10).index


CLASS FOR EFFICIENT FRONTIER USING MONTE CARLO METHOD

In [ ]:
class EfficientFrontier:
    def __init__(self,returns, assets):
        self.returns = returns
        self.assets = assets

    def Calc(self):
        self.tbl = pd.DataFrame(index = self.assets)
        self.tbl['Mean'] = self.returns.mean()
        self.tbl['Variance'] = self.returns.var()
        return self.tbl.T

   #CORRELATION
    def correlation(self):
        self.tbl = pd.DataFrame(index = self.assets)
        self.corr = self.returns.corr()
        return self.corr

    #PORTFOLIO RETURN
    def PortfolioReturn(self, weight):
        return np.sum(self.returns.mean() * weight) * 252

    #VOLATILITY
    def PortfolioVolatility(self, weight):
        return np.sqrt(np.dot(weight.T, np.dot(self.returns.cov() * 252, weight)))

    #MONTE CARLO
    def MonteCarlo(self, simulations = 100000):
        returnCol = []
        volatilityCol = []
        weightsCol = []
        sharperatioCol = []

        for i in range (simulations):
            weights = np.random.random(len(stocks))
            weights /= np.sum(weights)
            weightsCol.append(weights)
            port_return = self.PortfolioReturn(weights)
            port_volatility = self.PortfolioVolatility(weights)
            returnCol.append(port_return)
            volatilityCol.append(port_volatility)
            sharperatioCol.append((port_return - 0.01) / port_volatility)

        returnCol = np.array(returnCol)
        volatilityCol = np.array(volatilityCol)
        sharperatioCol = np.array(sharperatioCol)

        max_sharpe_idx = sharperatioCol.argmax()
        optimal_weights = weightsCol[max_sharpe_idx]

        return returnCol, volatilityCol, sharperatioCol, optimal_weights

#LINEAR REGRESSION
    def sharpeMinzd(self, weight):
        return -(self.PortfolioReturn(weight)-.02) / self.PortfolioVolatility(weight)

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for x in range(len(stocks)))
eWV = np.array(len(stocks) * [1.0 / len(stocks),])

ef = EfficientFrontier(returns=returns, assets=stocks)

sharpeOptimized = sco.minimize(ef.sharpeMinzd,
                             eWV,
                             method = 'SLSQP',
                             bounds = bounds,
                             constraints = constraints)
volOptimized = sco.minimize(ef.PortfolioVolatility,
                            eWV,
                            method = 'SLSQP',
                            bounds = bounds,
                            constraints = constraints)

treturns = np.linspace(min(returnCol), max(returnCol), 50)
tvolatility = []

for treturn in treturns:
    cons = ({'type': 'eq', 'fun': lambda x: ef.PortfolioReturn(x) - treturn},
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    res = sco.minimize(ef.PortfolioVolatility, eWV, method='SLSQP', bounds=bounds, constraints=cons)
    tvolatility.append(res['fun'])

tvolatility = np.array(tvolatility)